In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np

from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
import requests
import json
from xml.etree import ElementTree as ET

In [2]:
import requests
from xml.etree import ElementTree as ET

def fetch_gene_info(chromosome, start, end):
    query_xml = f"""
    <Query virtualSchemaName="default" formatter="TSV" header="0" uniqueRows="1" count="" datasetConfigVersion="0.6">
        <Dataset name="hsapiens_gene_ensembl" interface="default">
            <Filter name="chromosome_name" value="{chromosome}"/>
            <Filter name="start" value="{start}"/>
            <Filter name="end" value="{end}"/>
            <Attribute name="ensembl_gene_id"/>
            <Attribute name="external_gene_name"/>
            <Attribute name="description"/>
            <Attribute name="start_position"/>
            <Attribute name="end_position"/>            
            <Attribute name="strand"/>
        </Dataset>
    </Query>
    """

    biomart_url = "http://www.ensembl.org/biomart/martservice?query="
    response = requests.get(biomart_url + query_xml.strip())

    if response.status_code != 200:
        raise Exception(f"Error fetching data from BioMart: {response.text}")

    genes = [line.split("\t") for line in response.text.strip().split("\n")]
    print(genes)
    return genes

# chrom = "17"
# start = 45620566
# end = 46418024

# genes = fetch_gene_info(chrom, start, end)
# print("Genes in the specified region:")
# for gene in genes:
#     print(gene)
    # gene_id, gene_name, gene_start, gene_end, gene_description, gene_strand = gene
    # strand = "forward" if gene_strand == "1" else "reverse"
    # print(f"Gene ID: {gene_id}, Gene Name: {gene_name}, Start: {gene_start}, End: {gene_end}, Strand: {strand}, Description: {gene_description}")

In [3]:
split_race_region={'ACB': {'chr3': [[98029388, 98331063]],
  'chr4': [[8996392, 9139446]],
  'chr5': [[34277945, 34574089]]},
 'ASW': {'chr1': [[248297800, 248584618]]},
 'BEB': {'chr13': [[55108214, 55447878]]},
 'CDX': {'chr7': [[64033450, 64433853]], 'chr15': [[24061424, 24264850]]},
 'CEU': {'chr17': [[45825257, 46352295]]},
 'CHB': {},
 'CHS': {},
 'CLM': {'chr3': [[95042564, 95345546],
   [98029388, 98261301],
   [163846034, 164208695]],
  'chr17': [[45825257, 46352295]]},
 'ESN': {'chr1': [[248297800, 248584618]], 'chr5': [[34277945, 34574089]]},
 'FIN': {},
 'GBR': {'chr1': [[227003796, 227338558]],
  'chr3': [[98029388, 98261301]],
  'chr17': [[45825257, 46352295]]},
 'GIH': {'chr3': [[98029388, 98331063]], 'chr14': [[19806333, 20078222]]},
 'GWD': {'chr1': [[248297800, 248584618]],
  'chr5': [[34277945, 34574089]],
  'chr7': [[11926418, 12230106]]},
 'IBS': {'chr1': [[227003796, 227338558]], 'chr17': [[45825257, 46352295]]},
 'ITU': {'chr17': [[52570659, 52892893]]},
 'JPT': {},
 'KHV': {},
 'LWK': {'chr4': [[8996392, 9139446]]},
 'MSL': {'chr5': [[34231981, 34574089]]},
 'MXL': {'chr1': [[227003796, 227338558]], 'chr11': [[124162560, 124482002]]},
 'PEL': {'chr3': [[98029388, 98331063]], 'chr11': [[23181031, 23574546]]},
 'PJL': {'chr3': [[163846034, 164208695]]},
 'PUR': {'chr3': [[98029388, 98331063]], 'chr17': [[45825257, 46352295]]},
 'STU': {'chr3': [[98029388, 98293778]], 'chr13': [[55108214, 55447878]]},
 'TSI': {'chr7': [[67111221, 67437767]], 'chr17': [[45825257, 46352295]]},
 'YRI': {'chr1': [[227003796, 227338558], [248297800, 248584618]],
  'chr3': [[166091993, 166408110]],
  'chr5': [[34277945, 34574089]]}}

In [4]:
def generate_gene_info(gene_list_one_record,chrnum):
    result=[]
    result.append(gene_list_one_record[0])
    result.append(gene_list_one_record[1])
    s=gene_list_one_record[3]
    e=gene_list_one_record[4]
    
    pos="chr"+chrnum+":"+str(s)+"-"+str(e)
    result.append(pos)
    result.append(gene_list_one_record[5])
    result.append(gene_list_one_record[2])
    
    return result

In [5]:
import re

def extract_numbers(input_string):
    return re.findall(r'\d+', input_string)

In [6]:
def ifgeneinside_region(complementary_region,gene_region):
    
    s=int(gene_region[0])
    e=int(gene_region[1])
     
    if (complementary_region[0]<=s) and complementary_region[1]>=e:
        return True
    else:
        return False

In [7]:

allrecords=[]

#non_mhc
for race,split_ in split_race_region.items():
    if race!="All":
        for chr,arr in split_.items():
            chr=extract_numbers(chr)[0]
            print(chr)
            for a in arr:
                gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                if gene_list!=[['']]:
                    for gene_list_record in gene_list:
                        if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                            onerecord=[race,0]
                            onerecord+=["chr"+chr+":"+str(a[0])+"-"+str(a[1])]
                            
                            onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum=chr)
                            allrecords.append(onerecord)
                        else:
                            continue
                
    else:
        for chr,arr in split_.items():
            chr=extract_numbers(chr)[0]
            for a in arr:
                gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                if gene_list!=[['']]:
                    for gene_list_record in gene_list:
                        if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                            onerecord=["All",0]
                            onerecord+=["chr"+chr+":"+str(a[0])+"-"+str(a[1])]
                            
                            onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum=chr)
                            allrecords.append(onerecord)
                        else:
                            continue

3
[['ENSG00000183185', 'GABRR3', 'gamma-aminobutyric acid type A receptor subunit rho3 [Source:HGNC Symbol;Acc:HGNC:17969]', '97985102', '98035315', '-1'], ['ENSG00000250750', 'OR5BM1P', 'olfactory receptor family 5 subfamily BM member 1 pseudogene [Source:HGNC Symbol;Acc:HGNC:14835]', '98053374', '98053739', '1'], ['ENSG00000213439', 'OR5AC1', 'olfactory receptor family 5 subfamily AC member 1 (gene/pseudogene) [Source:HGNC Symbol;Acc:HGNC:15047]', '98064472', '98065396', '1'], ['ENSG00000196578', 'OR5AC2', 'olfactory receptor family 5 subfamily AC member 2 [Source:HGNC Symbol;Acc:HGNC:15431]', '98087173', '98088102', '1'], ['ENSG00000249225', '', 'novel transcript, antisense to OR5H1, OR5H14, OR5H15', '98099908', '98148134', '-1'], ['ENSG00000290819', '', 'novel transcript', '98100793', '98105672', '1'], ['ENSG00000251090', 'OR5AC4P', 'olfactory receptor family 5 subfamily AC member 4 pseudogene [Source:HGNC Symbol;Acc:HGNC:31284]', '98104754', '98105672', '1'], ['ENSG00000248365', '

In [8]:

mhc_race_positions_dict={'ACB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
'All':{'mhc':[[29720403, 29913914], [30994370, 31528792], [32212726, 32882258]]},
 'ASW': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'BEB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CDX': {'mhc': [[30994370, 31528792], [32212726, 32923168]]},
 'CEU': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'CHB': {'mhc': [[29720403, 29913914],
   [29939668, 30154438],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CHS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32397207, 32882258]]},
 'CLM': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ESN': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32423532, 32882258]]},
 'FIN': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GBR': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GIH': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'GWD': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'IBS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ITU': {'mhc': [[29720403, 30011739],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'JPT': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'KHV': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'LWK': {'mhc': [[29720403, 30011739],
   [31052133, 31528792],
   [32397207, 32923168]]},
 'MSL': {'mhc': [[29720403, 30011739],
   [30959575, 31528792],
   [32212726, 32882258]]},
 'MXL': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'PEL': {'mhc': [[29720403, 29913914],
   [30959575, 31528792],
   [32288923, 32882258]]},
 'PJL': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'PUR': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'STU': {'mhc': [[29720403, 29913914],
   [30994370, 31577278],
   [32212726, 32882258]]},
 'TSI': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'YRI': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]}}

In [9]:
len(allrecords)

424

In [10]:

for race, split_ in mhc_race_positions_dict.items():
    if split_!={}:
        if race!="All":
            for chr,arr in split_.items():
                chr="6" 
                     
                for a in arr:
                    gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                    if gene_list!=[['']]:
                        for gene_list_record in gene_list:
                            if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                                onerecord=[race,1]
                                onerecord+=["chr"+chr+":"+str(a[0])+"-"+str(a[1])]
                                
                                onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum="6")
                                allrecords.append(onerecord)   
                            else:
                                continue                
        else:
            for chr,arr in split_.items():
                chr="6"
                for a in arr:
                    gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                    if gene_list!=[['']]:
                        for gene_list_record in gene_list:
                            if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                                onerecord=["All",1]
                                onerecord+=["chr"+chr+":"+str(a[0])+"-"+str(a[1])]
                                
                                onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum="6")
                                allrecords.append(onerecord)    
                            else:
                                continue

[['ENSG00000204642', 'HLA-F', 'major histocompatibility complex, class I, F [Source:HGNC Symbol;Acc:HGNC:4963]', '29722775', '29738528', '1'], ['ENSG00000225864', '', 'HLA complex group 4 pseudogene 11', '29722981', '29723971', '-1'], ['ENSG00000214922', 'HLA-F-AS1', 'HLA-F antisense RNA 1 [Source:HGNC Symbol;Acc:HGNC:26645]', '29726601', '29749049', '-1'], ['ENSG00000239257', 'RPL23AP1', 'ribosomal protein L23a pseudogene 1 [Source:HGNC Symbol;Acc:HGNC:10318]', '29726669', '29727139', '-1'], ['ENSG00000273340', 'MICE', 'MHC class I polypeptide-related sequence E (pseudogene) [Source:HGNC Symbol;Acc:HGNC:7094]', '29741731', '29748969', '-1'], ['ENSG00000227758', 'HCG9P5', 'HLA complex group 9 pseudogene 5 [Source:HGNC Symbol;Acc:HGNC:30983]', '29748289', '29748513', '1'], ['ENSG00000199290', 'Y_RNA', 'Y RNA [Source:RFAM;Acc:RF00019]', '29750371', '29750472', '1'], ['ENSG00000235821', 'IFITM4P', 'interferon induced transmembrane protein 4 pseudogene [Source:HGNC Symbol;Acc:HGNC:21669]',

In [11]:
len(allrecords)

3324

In [12]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    #dataframe(dictforDF).to_csv("csv2_0721_test.csv")
    return dataframe(dictforDF)


In [13]:
dfnew=generatedf(columns=["Race","isMHC","Position of disassortative mating region","Gene ID","Gene name","Position of gene","is_complement","Gene description"],allrecords=allrecords)

In [14]:
dfnew['is_complement'] = dfnew['is_complement'].apply(lambda x:  "F" if x == "1" else "R")
# dfnew.to_csv("csv2_05_11new.csv")

In [15]:
dfnew

,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description
0,ACB,0,chr3:98029388-98331063,ENSG00000250750,OR5BM1P,chr3:98053374-98053739,F,olfactory receptor family 5 subfamily BM membe...
1,ACB,0,chr3:98029388-98331063,ENSG00000213439,OR5AC1,chr3:98064472-98065396,F,olfactory receptor family 5 subfamily AC membe...
2,ACB,0,chr3:98029388-98331063,ENSG00000196578,OR5AC2,chr3:98087173-98088102,F,olfactory receptor family 5 subfamily AC membe...
3,ACB,0,chr3:98029388-98331063,ENSG00000249225,,chr3:98099908-98148134,R,"novel transcript, antisense to OR5H1, OR5H14, ..."
4,ACB,0,chr3:98029388-98331063,ENSG00000290819,,chr3:98100793-98105672,F,novel transcript
...,...,...,...,...,...,...,...,...
3319,YRI,1,chr6:32212726-32882258,ENSG00000204264,PSMB8,chr6:32840717-32844679,R,proteasome 20S subunit beta 8 [Source:HGNC Sym...
3320,YRI,1,chr6:32212726-32882258,ENSG00000204261,PSMB8-AS1,chr6:32844078-32846500,F,PSMB8 antisense RNA 1 (head to head) [Source:H...
3321,YRI,1,chr6:32212726-32882258,ENSG00000240065,PSMB9,chr6:32844136-32859851,F,proteasome 20S subunit beta 9 [Source:HGNC Sym...
3322,YRI,1,chr6:32212726-32882258,ENSG00000168394,TAP1,chr6:32845209-32853816,R,"transporter 1, ATP binding cassette subfamily ..."


In [16]:
dfnew.to_csv("tables3_01part.csv")